In [62]:
import torch
from torch.autograd import Variable
import torch.autograd
import torch.nn as nn
import torch.nn.functional as F
import data_feeder as df
import numpy



class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        pretrained = False;
        if (pretrained == True):
            self.model = torchvision.models.vgg16(pretrained=True)
            for param in self.model.parameters():
                param.requires_grad = False
            self.model.fc = nn.Linear(128, 1) 
        else:
            self.model = nn.Sequential(
                    nn.Conv2d(3, 8, 5, stride = 2), 
                    nn.BatchNorm2d(8)
                        ,nn.ReLU()
                        ,nn.Conv2d(8, 16, 5, stride = 2)
                        ,nn.BatchNorm2d(16)
                            ,nn.ReLU()
                        ,nn.Conv2d(16, 32, 5)
                            ,nn.BatchNorm2d(32)
                            ,nn.ReLU()
                            ,nn.Conv2d(32, 48, 5)
                            ,nn.BatchNorm2d(48)
                            ,nn.ReLU()
                            ,nn.Conv2d(48, 1, 5)
               #                 ,nn.BatchNorm2d(24)
                )
            
           # self.fc1 = nn.Linear(24, 64)
           # self.fc2 = nn.Linear(64, 1)
              #                  ,nn.Linear(48 * 48 * 24, 128)
             #                       ,nn.BatchNorm1d(128)
            #                        ,nn.ReLU()
           #                         ,nn.Linear(128, 1)
                
    
    def forward(self, input_data):
        return self.model(input_data)
    
class Solver():
    def __init__(self):
        super(Solver, self).__init__()
        
    def loss(self, pred, labels):
        return (pred - labels).pow(2).sum() / len(labels)

    def train(self, train_data, num_epochs = 10, learning_rate = 1e-2, batch_size = 16, pretrained = False):

            train_loader = torch.utils.data.DataLoader(dataset = train_data,

                                               batch_size=batch_size, 

                                               shuffle=True)
            print(len(train_loader))
            cnn = Model()
            optimizer = torch.optim.Adam(cnn.parameters(), lr = learning_rate)

            for i in range(num_epochs):
                for j, (images, labels) in enumerate(train_loader):
                    images = Variable(images.float())
                    labels = Variable(labels.float())
                    optimizer.zero_grad()
                    pred = cnn(images)
                    loss = self.loss(pred, labels)
                    #loss = self.loss(pred, labels)
                    cnn.zero_grad()
                    loss.backward()
                    optimizer.step()
                       
                    if (j+1) % 10 == 0:

                        print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 

                           %(i+1, num_epochs, j+1, len(train_loader)//batch_size, loss.data[0]))
                        
train_data, val_data = df.getDrivingData('race1516207298.txt')

solver = Solver()
solver.train(train_data)         
    
        
    
    
    
    

25
Epoch [1/10], Iter [10/1] Loss: 4.2847
Epoch [1/10], Iter [20/1] Loss: 0.5129
Epoch [2/10], Iter [10/1] Loss: 0.1219
Epoch [2/10], Iter [20/1] Loss: 0.4458
Epoch [3/10], Iter [10/1] Loss: 0.2145
Epoch [3/10], Iter [20/1] Loss: 0.1343
Epoch [4/10], Iter [10/1] Loss: 0.1587
Epoch [4/10], Iter [20/1] Loss: 0.1037
Epoch [5/10], Iter [10/1] Loss: 0.1032
Epoch [5/10], Iter [20/1] Loss: 0.2043
Epoch [6/10], Iter [10/1] Loss: 0.1288
Epoch [6/10], Iter [20/1] Loss: 0.0925
Epoch [7/10], Iter [10/1] Loss: 0.0953
Epoch [7/10], Iter [20/1] Loss: 0.0691
Epoch [8/10], Iter [10/1] Loss: 0.0657
Epoch [8/10], Iter [20/1] Loss: 0.0905
Epoch [9/10], Iter [10/1] Loss: 0.0662
Epoch [9/10], Iter [20/1] Loss: 0.1210
Epoch [10/10], Iter [10/1] Loss: 0.0451
Epoch [10/10], Iter [20/1] Loss: 0.1440
